In [3]:
def shift_and_print(iv):
    while iv:
        ot = 1 & iv # save the rightmost bit
        iv >>= 1 # shift
        print(ot)

In [2]:
def shift_and_yield(iv):
    while iv:
        ot = 1 & iv # save the rightmost bit
        iv >>= 1 # shift
        yield ot

In [4]:
shift_and_print(int(hex(0x63), 16))


1
1
0
0
0
1
1


In [5]:
shift_and_print(int('1001110', 2))


0
1
1
1
0
0
1


In [6]:
print('Output in one line = ',
      ''.join(str(output) for output in shift_and_yield(int('1101001', 2))))

Output in one line =  1001011


## Linear Feedback Shift Register



In [21]:
def lfsr_generate(seed, mask, n):
    seed_int = int(seed, 2)
    mask_int = int(mask, 2)
    nbits = len(seed)

    state = seed_int
    while n > 0:
        output =  1 & state # get the most right bit

        _mask, _state, new_bit = mask_int, state, 0 # store the values of mask_int and state, also set the new bit to zero, which will be later modified based on the mask and seed value
        while _mask:
            new_bit ^= (1 & _mask) * (1 & _state) # bit by bit xor with the current state, based on the mask
            _mask >>= 1 # shift to the right
            _state >>= 1 # shift to the right

        state = (state >> 1) | (new_bit << (nbits - 1)) # Shift to the right by one, and put (concatenate) the new bit in the msb

        yield output, state
        n -= 1

In [22]:
seed = '0001'       # secret initial seed
mask = '0101'       # e.g. 101 means --> x^2 + 1
samples = 20        # required number of random bits

In [23]:
key = lfsr_generate(seed, mask, samples)
key_str = ''.join(str(x) for x, _ in key)
key_hex = hex(int(key_str, 2))[2:]

In [24]:
print("[x] output lfsr sequence = ", key_str)
# For the above initial setting the output should be --> [x] output lfsr sequence =  10001010001010001010
assert key_str == '10001010001010001010'

[x] output lfsr sequence =  10001010001010001010


## Encryption and Decryption using LFSR



In [25]:
def encrypt(plain, key: str):
    # Your code Here
    # For encryption take two digits of the key (entered as hexa), convert them to integer, then XOR them with the ascii code of charachter, convert the result back to symnbol, to be repeated for all string
    key_int = int(key[:4], 16)
    result = "".join([chr(ord(ch) ^ key_int) for ch in plain])
    return result

In [26]:
def decrypt(cipher, key):
    # Your code here
    # reverse the encryption process. The code is identical to the encryption part.
    result = encrypt(cipher, key)
    return result

In [27]:
plain = 'something new'     # plaintext entered as string, e.g. 'something new'
secret_seed = '0001'        # secret initial seed
mask = '0101'               # e.g. 101 means --> x^2 + 1
samples = len(plain) * 8    # required number of random bits --> number of characters in plaintext multiplied by 8

In [31]:
key = lfsr_generate(secret_seed, mask, samples)
key_str = ''.join([str(b) for b, _ in key]) # Convert the key to a sequence of binary bits
# based on the above initial setting the output of key_str should be --> 10001010001010001010001010001010001010001010001010001010001010001010001010001010001010001010001010001010
key_str

'10001010001010001010001010001010001010001010001010001010001010001010001010001010001010001010001010001010'

In [35]:
key_hex = hex(int(key_str, 2))# Get the hexa value of the key
key_hex 

'0x8a28a28a28a28a28a28a28a28a'

In [36]:
cipher = encrypt(plain, key_hex)
print("[x] Ciphertext = ", cipher.encode('utf-8').hex())
# based on the above initial setting the output of hexa value of the encoded cipher should be --> c3b947c38fc3af5cc38ac3a346c385c2aa46c387c3bd

[x] Ciphertext =  c3b9c3a5c3a7c3afc3bec3a2c3a3c3a4c3adc2aac3a4c3afc3bd


In [37]:

re_plain = decrypt(cipher, key_hex)
print("[x] Recovered plaintext = ", re_plain)

[x] Recovered plaintext =  something new


## Cracking LFSR



In [38]:
from berlekampmassey import bm


poly = list(bm('10001010001010001010'))[-1]
print("[*] Recovering secret key  : ",poly[::-1])  # The recovered polynomial is our key

[*] Recovering secret key  :  [0, 1, 0, 1]


## ChaCha20



In [40]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
import os

def chacha20_encrypt(plaintext, key, nonce):
    cipher = Cipher(algorithms.ChaCha20(key, nonce), mode=None, backend=default_backend())
    encryptor = cipher.encryptor()
    return encryptor.update(plaintext)

def chacha20_decrypt(ciphertext, key, nonce):
    cipher = Cipher(algorithms.ChaCha20(key, nonce), mode=None, backend=default_backend())
    decryptor = cipher.decryptor()
    return decryptor.update(ciphertext)

# Generate a random 32-byte key and 16-byte nonce
key = os.urandom(32)  # 256-bit key
nonce = os.urandom(16)  # 128-bit nonce

# Example usage
plaintext = b"Secret message"
ciphertext = chacha20_encrypt(plaintext, key, nonce)
decrypted = chacha20_decrypt(ciphertext, key, nonce)

print(f"Plaintext: {plaintext}")
print(f"Ciphertext: {ciphertext.hex()}")
print(f"Decrypted: {decrypted}")

Plaintext: b'Secret message'
Ciphertext: 86509868cf4a2004b37c84046017
Decrypted: b'Secret message'
